In [3]:
pip install yahoo_fin

     |████████████████████████████████| 81 kB 4.2 MB/s 
     |████████████████████████████████| 83 kB 1.9 MB/s 
     |████████████████████████████████| 127 kB 42.0 MB/s 
     |████████████████████████████████| 111 kB 38.4 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=0df54927c1222d9f48b4dd5abde8dbce5cb1a45598c11e2d5ce7dc6d9cae2cd1
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=2d3fa115dd6f8a3df187deab69f7c3fb37c60de0339616cd35b325a6b2dc6ff6
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=a45bf061d3c4c1876034ebaa88b002deea026d25c4bb65ed98538c3970620235
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36ce

In [4]:
mkdir -p data results log


In [8]:
import tradealg

In [6]:

import imp


In [10]:
pwd

'/content'

In [8]:
imp.reload(tradealg)

<module 'tradealg' from '/content/tradealg.py'>

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from yahoo_fin import stock_info as si
from collections import deque

import os
import numpy as np
import pandas as pd
import random
import time
from tensorflow.keras.layers import LSTM
import matplotlib.pyplot as plt

In [76]:

def prepare():
    # set seed, so we can get the same results after rerunning several times
    np.random.seed(314)
    tf.random.set_seed(314)
    random.seed(314)


def shuffle_in_unison(a, b):
    # shuffle two arrays in the same way
    state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(state)
    np.random.shuffle(b)

def load_data(ticker, n_steps=50, scale=True, shuffle=True, lookup_step=1, split_by_date=True,
                test_size=0.2, feature_columns=['adjclose', 'volume', 'open', 'high', 'low']):
    """
    Loads data from Yahoo Finance source, as well as scaling, shuffling, normalizing and splitting.
    Params:
        ticker (str/pd.DataFrame): the ticker you want to load, examples include AAPL, TESL, etc.
        n_steps (int): the historical sequence length (i.e window size) used to predict, default is 50
        scale (bool): whether to scale prices from 0 to 1, default is True
        shuffle (bool): whether to shuffle the dataset (both training & testing), default is True
        lookup_step (int): the future lookup step to predict, default is 1 (e.g next day)
        split_by_date (bool): whether we split the dataset into training/testing by date, setting it 
            to False will split datasets in a random way
        test_size (float): ratio for test data, default is 0.2 (20% testing data)
        feature_columns (list): the list of features to use to feed into the model, default is everything grabbed from yahoo_fin
    """
    # see if ticker is already a loaded stock from yahoo finance
    if isinstance(ticker, str):
        # load it from yahoo_fin library
        df = si.get_data(ticker)
    elif isinstance(ticker, pd.DataFrame):
        # already loaded, use it directly
        df = ticker
    else:
        raise TypeError("ticker can be either a str or a `pd.DataFrame` instances")
    # this will contain all the elements we want to return from this function
    result = {}
    # we will also return the original dataframe itself
    result['df'] = df.copy()
    # make sure that the passed feature_columns exist in the dataframe
    for col in feature_columns:
        assert col in df.columns, f"'{col}' does not exist in the dataframe."
    # add date as a column
    if "date" not in df.columns:
        df["date"] = df.index
    if scale:
        column_scaler = {}
        # scale the data (prices) from 0 to 1
        for column in feature_columns:
            scaler = preprocessing.MinMaxScaler()
            df[column] = scaler.fit_transform(np.expand_dims(df[column].values, axis=1))
            column_scaler[column] = scaler
        # add the MinMaxScaler instances to the result returned
        result["column_scaler"] = column_scaler
    # add the target column (label) by shifting by `lookup_step`
    df['future'] = df['adjclose'].shift(-lookup_step)
    # last `lookup_step` columns contains NaN in future column
    # get them before droping NaNs
    last_sequence = np.array(df[feature_columns].tail(lookup_step))
    # drop NaNs
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=n_steps)
    for entry, target in zip(df[feature_columns + ["date"]].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == n_steps:
            sequence_data.append([np.array(sequences), target])
    # get the last sequence by appending the last `n_step` sequence with `lookup_step` sequence
    # for instance, if n_steps=50 and lookup_step=10, last_sequence should be of 60 (that is 50+10) length
    # this last_sequence will be used to predict future stock prices that are not available in the dataset
    last_sequence = list([s[:len(feature_columns)] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)
    # add to result
    result['last_sequence'] = last_sequence
    # construct the X's and y's
    X, y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        y.append(target)
    # convert to numpy arrays
    X = np.array(X)
    y = np.array(y)
    if split_by_date:
        # split the dataset into training & testing sets by date (not randomly splitting)
        train_samples = int((1 - test_size) * len(X))
        result["X_train"] = X[:train_samples]
        result["y_train"] = y[:train_samples]
        result["X_test"]  = X[train_samples:]
        result["y_test"]  = y[train_samples:]
        if shuffle:
            # shuffle the datasets for training (if shuffle parameter is set)
            shuffle_in_unison(result["X_train"], result["y_train"])
            shuffle_in_unison(result["X_test"], result["y_test"])
    else:    
        # split the dataset randomly
        result["X_train"], result["X_test"], result["y_train"], result["y_test"] = train_test_split(X, y, test_size=test_size, shuffle=shuffle)
    # get the list of test set dates
    dates = result["X_test"][:, -1, -1]
    # retrieve test features from the original dataframe
    result["test_df"] = result["df"].loc[dates]
    # remove duplicated dates in the testing dataframe
    result["test_df"] = result["test_df"][~result["test_df"].index.duplicated(keep='first')]
    # remove dates from the training/testing sets & convert to float32
    result["X_train"] = result["X_train"][:, :, :len(feature_columns)].astype(np.float32)
    result["X_test"] = result["X_test"][:, :, :len(feature_columns)].astype(np.float32)
    return result

def get_final_df(model, data, scale, lookup_step, trade):
    """
    This function takes the `model` and `data` dict to 
    construct a final dataframe that includes the features along 
    with true and predicted prices of the testing dataset
    """
    X_test = data["X_test"]
    y_test = data["y_test"]
    # perform prediction and get prices
    y_pred = model.predict(X_test)
    if scale:
        y_test = np.squeeze(data["column_scaler"]["adjclose"].inverse_transform(np.expand_dims(y_test, axis=0)))
        y_pred = np.squeeze(data["column_scaler"]["adjclose"].inverse_transform(y_pred))
    test_df = data["test_df"]
    # add predicted future prices to the dataframe
    test_df[f"adjclose_{lookup_step}"] = y_pred
    # add true future prices to the dataframe
    test_df[f"true_adjclose_{lookup_step}"] = y_test
    # sort the dataframe by date
    test_df.sort_index(inplace=True)
    final_df = test_df
    # add the buy profit column
    final_df["buy_profit"] = list(map(trade.buy_profit, 
                                    final_df["adjclose"], 
                                    final_df[f"adjclose_{lookup_step}"], 
                                    final_df[f"true_adjclose_{lookup_step}"])
                                    # since we don't have profit for last sequence, add 0's
                                    )
    # add the sell profit column
    final_df["sell_profit"] = list(map(trade.sell_profit, 
                                    final_df["adjclose"], 
                                    final_df[f"adjclose_{lookup_step}"], 
                                    final_df[f"true_adjclose_{lookup_step}"])
                                    # since we don't have profit for last sequence, add 0's
                                    )
    return final_df


class TradingResult:
    def __init__(self, model, prepdata, lossn):
        self.model = model
        self.pdata = prepdata
        self.data = prepdata.data
        self.LOSSN = lossn
        
    def predict(self):
        # retrieve the last sequence from data
        last_sequence = self.data["last_sequence"][-self.pdata.N_STEPS:]
        # expand dimension
        last_sequence = np.expand_dims(last_sequence, axis=0)
        # get the prediction (scaled from 0 to 1)
        prediction = self.model.predict(last_sequence)
        # get the price (by inverting the scaling)
        if self.pdata.SCALE:
            predicted_price = self.data["column_scaler"]["adjclose"].inverse_transform(prediction)[0][0]
        else:
            predicted_price = prediction[0][0]
        return predicted_price

    def eval(self, trade):
        # evaluate the model
        loss, mae = self.model.evaluate(self.data["X_test"], self.data["y_test"], verbose=0)
        # calculate the mean absolute error (inverse scaling)
        if self.pdata.SCALE:
            self.mean_absolute_error = self.data["column_scaler"]["adjclose"].inverse_transform([[mae]])[0][0]
        else:
            self.mean_absolute_error = mae
            
        # get the final dataframe for the testing set
        final_df = get_final_df(self.model, self.data, self.pdata.SCALE, self.pdata.LOOKUP_STEP, trade)
        # predict the future price
        self.future_price = self.predict()
        # we calculate the accuracy by counting the number of positive profits
        self.accuracy_score = (len(final_df[final_df['sell_profit'] > 0]) + len(final_df[final_df['buy_profit'] > 0])) / len(final_df)
        # calculating total buy & sell profit
        self.total_buy_profit  = final_df["buy_profit"].sum()
        self.total_sell_profit = final_df["sell_profit"].sum()
        # total profit by adding sell & buy together
        self.total_profit = self.total_buy_profit + self.total_sell_profit
        # dividing total profit by number of testing samples (number of trades)
        self.profit_per_trade = self.total_profit / len(final_df)
        self.final_df = final_df
        self.loss = loss

    def print(self):
        # printing metrics
        print(f"Ticker {self.pdata.ticker}")
        print(f"Future price after {self.pdata.LOOKUP_STEP} days is {self.future_price:.2f}$")
        print(f"{self.LOSSN}_loss:", self.loss)
        print("Mean Absolute Error:", self.mean_absolute_error)
        print("Accuracy score:", self.accuracy_score)
        print("Total buy profit:", self.total_buy_profit)
        print("Total sell profit:", self.total_sell_profit)
        print("Total profit:", self.total_profit)
        print("Profit per trade:", self.profit_per_trade)
        
class PreparedData:
    def __init__(self, ticker):
        # Window size or the sequence length
        self.N_STEPS = 50
	# Lookup step, 1 is the next day
        self.LOOKUP_STEP = 15
        # whether to scale feature columns & output price as well
        self.SCALE = True
        self.scale_str = f"sc-{int(self.SCALE)}"
        # whether to shuffle the dataset
        self.SHUFFLE = True
        self.shuffle_str = f"sh-{int(self.SHUFFLE)}"
        # whether to split the training/testing set by date
        self.SPLIT_BY_DATE = False
        self.split_by_date_str = f"sbd-{int(self.SPLIT_BY_DATE)}"
        # test ratio size, 0.2 is 20%
        self.TEST_SIZE = 0.2
        # features to use
        self.FEATURE_COLUMNS = ["adjclose", "volume", "open", "high", "low"]
        self.ticker = ticker
        # date now
        self.date_now = time.strftime("%Y-%m-%d")        
        self.ticker_data_filename = os.path.join("data", f"{self.ticker}_{self.date_now}.csv")
        self.data_prefix = f"{self.date_now}_{self.ticker}-{self.shuffle_str}-{self.scale_str}-{self.split_by_date_str}-seq-{self.N_STEPS}-step-{self.LOOKUP_STEP}"
        
    def prepare(self,  df):
        # load the data
        self.data = load_data(df, self.N_STEPS, scale=self.SCALE, split_by_date=self.SPLIT_BY_DATE, 
                shuffle=self.SHUFFLE, lookup_step=self.LOOKUP_STEP, test_size=self.TEST_SIZE, 
                feature_columns=self.FEATURE_COLUMNS)

        # save the dataframe
        self.data["df"].to_csv(self.ticker_data_filename)


def fetch_data(ticker):
# see if ticker is already a loaded stock from yahoo finance
    if isinstance(ticker, str):
        # load it from yahoo_fin library
        df = si.get_data(ticker)
    elif isinstance(ticker, pd.DataFrame):
        # already loaded, use it directly
        df = ticker
    else:
        raise TypeError("ticker can be either a str or a `pd.DataFrame` instances")
    return df


def create_model(sequence_length, n_features, units=256, cell=LSTM, n_layers=2, dropout=0.3,
                loss="mean_absolute_error", optimizer="rmsprop", bidirectional=False):
    model = Sequential()
    for i in range(n_layers):
        if i == 0:
            # first layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True), batch_input_shape=(None, sequence_length, n_features)))
            else:
                model.add(cell(units, return_sequences=True, batch_input_shape=(None, sequence_length, n_features)))
        elif i == n_layers - 1:
            # last layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=False)))
            else:
                model.add(cell(units, return_sequences=False))
        else:
            # hidden layers
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True)))
            else:
                model.add(cell(units, return_sequences=True))
        # add dropout after each layer
        model.add(Dropout(dropout))
    model.add(Dense(1, activation="linear"))
    model.compile(loss=loss, metrics=["mean_absolute_error"], optimizer=optimizer)
    return model


import os
import time
from tensorflow.keras.layers import LSTM

EPOCHS = 20

class RNNModel:
    def __init__(self):
        self.date_now = time.strftime("%Y-%m-%d")
        ### model parameters
        self.N_LAYERS = 2
        # LSTM cell
        self.CELL = LSTM
        # 256 LSTM neurons
        self.UNITS = 256
        # 40% dropout
        self.DROPOUT = 0.4
        # whether to use bidirectional RNNs
        self.BIDIRECTIONAL = False
        ### training parameters
        # mean absolute error loss
        # LOSS = "mae"
        # huber loss
        self.LOSS = "huber_loss"
        self.OPTIMIZER = "adam"
        self.BATCH_SIZE = 64
        self.EPOCHS = EPOCHS # 500

    def create(self, prepdata):
        # model name to save, making it as unique as possible based on parameters
        self.model_name = f"{prepdata.data_prefix}-model-{self.LOSS}-{self.OPTIMIZER}-{self.CELL.__name__}-layers-{self.N_LAYERS}-units-{self.UNITS}"
        if self.BIDIRECTIONAL:
            self.model_name += "-b"

        # create these folders if they does not exist
        if not os.path.isdir("results"):
            os.mkdir("results")
        if not os.path.isdir("logs"):
            os.mkdir("logs")
        if not os.path.isdir("data"):
            os.mkdir("data")

        self.model = create_model(prepdata.N_STEPS, len(prepdata.FEATURE_COLUMNS), loss=self.LOSS, units=self.UNITS, cell=self.CELL, n_layers=self.N_LAYERS,
                    dropout=self.DROPOUT, optimizer=self.OPTIMIZER, bidirectional=self.BIDIRECTIONAL)
        # some tensorflow callbacks
        self.checkpointer = ModelCheckpoint(os.path.join("results", self.model_name + ".h5"), save_weights_only=True, save_best_only=True, verbose=1)
        self.tensorboard = TensorBoard(log_dir=os.path.join("logs", self.model_name))

    def train(self, data):
        # train the model and save the weights whenever we see 
        # a new optimal model using ModelCheckpoint
        history = self.model.fit(data["X_train"], data["y_train"],
                                 batch_size=self.BATCH_SIZE,
                                 epochs=self.EPOCHS,
                                 validation_data=(data["X_test"], data["y_test"]),
                                 callbacks=[self.checkpointer, self.tensorboard],
                                 verbose=1)
        self.load()

    def load(self):
        # load optimal model weights from results folder
        model_path = os.path.join("results", self.model_name) + ".h5"
        self.model.load_weights(model_path)



def runModel(ticker, modifier, trading, do_train=True):
    data = fetch_data(ticker)

    data = modifier.change_data(data)
    
    pdata = PreparedData(ticker)
    modifier.change_prep(pdata)
        
    pdata.prepare(data)
    
    mod = RNNModel()

    modifier.change_model(mod)
    mod.create(pdata)
    if not do_train:
        # TODO train only when not already trained
        mod.load()
    else:
        mod.train(pdata.data)
    res = TradingResult(mod.model, pdata, mod.LOSS)
    res.eval(trading)
    res.print()
    modifier.print(res)
    #df.set_index(['Ticker', 'Name'])
    return {'Ticker': ticker, 'Name': modifier.name, 'Buy': res.total_buy_profit,
               'Sell': res.total_sell_profit, 'Total': res.total_profit}
              


class NormalTrading:
    buy_profit  = lambda current, pred_future, true_future: true_future - current if pred_future > current else 0
    sell_profit = lambda current, pred_future, true_future: current - true_future if pred_future < current else 0

class NoModifier:
    name = 'Original'
    
    def change_prep(self, pdata):
        pass

    def change_model(self, mod):
        pass

    def change_data(self, data):
        return data

    def print(self, res):
      pass


class AddDay(NoModifier):
    def __init__(self):
        self.name = 'Day'
    
    def change_prep(self, pdata):
        pdata.FEATURE_COLUMNS = pdata.FEATURE_COLUMNS + ['day']
        pdata.ticker_data_filename += '-withday'
        pdata.data_prefix += '-withday'
        pass


    def change_data(self, data):
        # add date as a column
        if "date" not in data.columns:
            data["date"] = data.index

        df = data.apply(lambda row: row.date.timetuple().tm_yday, axis = 1)
        return df.to_frame('day').join(data)

class AddDayMonth(NoModifier):
    def __init__(self):
      self.name = 'DayMon'
    
    def change_prep(self, pdata):
        pdata.FEATURE_COLUMNS = pdata.FEATURE_COLUMNS + ['mday', 'month']
        pdata.ticker_data_filename += '-wdm'
        pdata.data_prefix += '-wdm'
        pass


    def change_data(self, data):
        # add date as a column
        if "date" not in data.columns:
            data["date"] = data.index

        dfd = data.apply(lambda row: row.date.timetuple().tm_mday, axis = 1)
        dfm = data.apply(lambda row: row.date.timetuple().tm_mon, axis = 1)
        df = dfd.to_frame('mday').join(data)
        return dfm.to_frame('month').join(df)
            

def getStatFrame():
    return pd.DataFrame(columns=['Ticker', 'Name', 'Buy', 'Sell', 'Total'])
    
def runTicker(ticker, models = [NoModifier], df=getStatFrame()):
    for model in models:
        prepare()
        result = runModel(ticker, model, NormalTrading, True)
        df = df.append(result, ignore_index=True)
    return df



def runTickers(tickers, models):
    df = getStatFrame()
    for ticker in tickers:
        df = runTicker(ticker, models, df)
    print(df)
    
# Todo: Combined model
# Model based on rate of return from previous day
# fix high, low based on adjusted price
# Incremental data load and training


# def combinedModel(tickers):
#     pdatas = []
#     for ticker in tickers:
#         data = fetch_data(ticker)
#         pdata = PrepareData()
#         pdata.prepare(ticker, data)
#         pdatas.add(pdata)

#     combine = ProcessedData()
#     for pdata in pdatas:
#         combined.data['X_train'].append(ticker, pdata.data)
#         combined.data['X_test'].append(ticker, pdata.data)
#         y_train
#         y_test
#         combined.date_now = ''
#         combined.ticker_data_file_name = ''

#     model.fit
    
#         combined.result['column_scaler] = virtual column scaler
        
        
        


In [137]:
class AddMA(NoModifier):
  def __init__(self, num, col='adjclose'):
    self.period = num
    self.colname = f"ma-{col}-{num}"
    self.col = col
    self.name = self.colname

  def change_prep(self, pdata):
        pdata.FEATURE_COLUMNS = pdata.FEATURE_COLUMNS + [self.colname]
        pdata.ticker_data_filename += f"-w{self.colname}"
        pdata.data_prefix += f"-w{self.colname}"
  
  def change_data(self, data):
      df = (data[self.col]
            .rolling(window=self.period)
            .mean()
            .to_frame(self.colname)
            .dropna())
      return df.join(data)


In [139]:
class CropData(NoModifier):
    def __init__(self, num):
      self.num = num
      self.name = f"Crop{num}"

    def change_data(self, data):
      return data.tail(self.num)

In [132]:
class FeatureSeq(NoModifier):
  def __init__(self, classes):
    self.name = 'seq'
    self.modifiers = classes 
    for cls in classes:
      self.name += cls.name
    
  def change_prep(self, pdata):
    for cls in self.modifiers:
      cls.change_prep(pdata)
    
  def change_model(self, mod):
    for obj in self.modifiers:
      obj.change_model(mod)

  def change_data(self, data):
    for cls in self.modifiers:
      data = cls.change_data(data)
    return data

  def print(self, res):
    for cls in self.modifiers:
      cls.print(res)

In [127]:
class RateReturnOnly(NoModifier):

  def __init__(self):
    self.name = 'RROnly'
    # =['adjclose', 'volume', 'open', 'high', 'low']

  def change_prep(self, pdata):
        pdata.ticker_data_filename += f"-w{self.name}"
        pdata.data_prefix += f"-w{self.name}"
  
  def change_data(self, data):
    sdata = data.rolling(200).mean()
    self.lastref = sdata.tail(1).adjclose.item()
    newdata = data
    newdata['adjclose'] = (data['adjclose']-sdata['adjclose'])/sdata['adjclose']
    newdata['volume'] = (data['volume']-sdata['volume'])/sdata['volume']
    newdata['open'] = data['open']/data['adjclose']
    newdata['high'] = data['high']/data['adjclose']-1
    newdata['low'] = 1-data['low']/data['adjclose']
    newdata['close'] =  data['close']/data['adjclose']
    newdata['ticker'] = data['ticker']
    newdata.dropna(inplace=True)
    return newdata.tail(2000)
  
  def print(self, res):
      print(f"Last 200 dma {self.lastref}")
      print(f"Future price {self.lastref * (1 + res.future_price)}")



In [112]:
testdata = fetch_data('NET')

In [114]:
testdata.tail(1).adjclose.item()

100.29000091552734

In [116]:
obj = RateReturnOnly()
testdf = obj.change_data(testdata)
testdata.tail(6), testdf.tail(6)
print(obj.lastref)


100.29000091552734


In [62]:
mod = FeatureSeq([AddDayMonth, AddMA(200)])
newdata = mod.change_data(testdata)
newdata.tail(10)
tpdata = PreparedData('AMZN')
tpdata.prepare(newdata)
mod.change_prep(tpdata)

In [123]:
tpdata.data["X_test"]

array([[[0.01989295, 0.05625344, 0.02001557, 0.01982755, 0.01990637],
        [0.01996543, 0.05777111, 0.01998347, 0.02013815, 0.02022604],
        [0.02030633, 0.04243005, 0.01997544, 0.02008771, 0.02026127],
        ...,
        [0.01871994, 0.03496349, 0.01874754, 0.01890108, 0.01882816],
        [0.01963258, 0.06682881, 0.01899633, 0.01949041, 0.01923723],
        [0.01935879, 0.05340659, 0.01943238, 0.01925415, 0.01929412]],

       [[0.20780095, 0.02570376, 0.20754449, 0.20590764, 0.20795225],
        [0.20786807, 0.00923943, 0.20882055, 0.20723496, 0.2092472 ],
        [0.2042175 , 0.03438929, 0.20627646, 0.20464669, 0.20554659],
        ...,
        [0.22386082, 0.06253583, 0.22222045, 0.22203197, 0.22289017],
        [0.21587518, 0.09654618, 0.21422972, 0.21561037, 0.21631792],
        [0.21518801, 0.02915477, 0.21505368, 0.21359815, 0.216047  ]],

       [[0.01914673, 0.0462426 , 0.01953672, 0.01935237, 0.01923452],
        [0.01967016, 0.04624937, 0.01896423, 0.01945856, 0.0

In [122]:
EPOCHS=20

In [140]:
#runTicker('AMZN', [NoModifier(), AddDayMonth(),AddMA(200)])
#runTicker('MSFT', [FeatureSeq([AddDayMonth(), AddMA(200), CropData(2000)])])
modlist = [FeatureSeq([AddDayMonth(), AddMA(200), CropData(2000)]), FeatureSeq([RateReturnOnly(), AddDayMonth()])]
runTicker('PTON', modlist)

Epoch 1/20
4/4 [==============================] - ETA: 0s - loss: 0.0751 - mean_absolute_error: 0.3083
Epoch 00001: val_loss improved from inf to 0.03123, saving model to results/2022-01-17_PTON-sh-1-sc-1-sbd-0-seq-50-step-15-wdm-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
4/4 [==============================] - 6s 419ms/step - loss: 0.0751 - mean_absolute_error: 0.3083 - val_loss: 0.0312 - val_mean_absolute_error: 0.2170
Epoch 2/20
4/4 [==============================] - ETA: 0s - loss: 0.0201 - mean_absolute_error: 0.1637
Epoch 00002: val_loss improved from 0.03123 to 0.01009, saving model to results/2022-01-17_PTON-sh-1-sc-1-sbd-0-seq-50-step-15-wdm-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
4/4 [==============================] - 0s 75ms/step - loss: 0.0201 - mean_absolute_error: 0.1637 - val_loss: 0.0101 - val_mean_absolute_error: 0.1176
Epoch 3/20
3/4 [=====================>........] - ETA: 0s - loss: 0.0198 - mean_absolute_error: 0.1639


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Epoch 1/20
4/4 [==============================] - ETA: 0s - loss: 0.0493 - mean_absolute_error: 0.2460
Epoch 00001: val_loss improved from inf to 0.02648, saving model to results/2022-01-17_PTON-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-model-huber_loss-adam-LSTM-layers-2-units-256.h5
4/4 [==============================] - 6s 415ms/step - loss: 0.0493 - mean_absolute_error: 0.2460 - val_loss: 0.0265 - val_mean_absolute_error: 0.1839
Epoch 2/20
4/4 [==============================] - ETA: 0s - loss: 0.0156 - mean_absolute_error: 0.1259
Epoch 00002: val_loss improved from 0.02648 to 0.00803, saving model to results/2022-01-17_PTON-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-model-huber_loss-adam-LSTM-layers-2-units-256.h5
4/4 [==============================] - 0s 76ms/step - loss: 0.0156 - mean_absolute_error: 0.1259 - val_loss: 0.0080 - val_mean_absolute_error: 0.1058
Epoch 3/20
3/4 [=====================>........] - ETA: 0s - loss: 0.0119 - mean_absolute_error: 0.1290
Epoch 00003: val_l

,Ticker,Name,Buy,Sell,Total
0,PTON,seqDayMonma-adjclose-200Crop2000,167.099991,249.450005,416.549995
1,PTON,seqRROnlyDayMon,0.699314,6.649239,7.348553
